In [ ]:
# if need be, manually add the local project root to PYTHONPATH and move working directories

import os
import sys

project = '/' # change to local project root
sys.path.append(project)
os.chdir(project)

In [ ]:
# dependencies

import numpy as np
import pandas as pd

import xfx.glm.binomial
import xfx.misc.plot

In [ ]:
# helper functions

def package_samples(samples, factor_names):

    rfx_samples, prec_samples = zip(*samples)
    rfx_samples = [np.array(samples_) for samples_ in zip(*rfx_samples)]
    prec_samples = np.array(prec_samples)
    return package_rfx_samples(rfx_samples, ['_const'] + factor_names), package_prec_samples(prec_samples, factor_names)

def package_rfx_samples(rfx_samples, factor_names):

    dfs = []
    for samples_, factor_name in zip(rfx_samples, factor_names):
        df_ = pd.DataFrame(samples_.T)
        df_.index = df_.index.rename('level')
        df_.columns = df_.columns.rename('iter')
        df_['factor'] = factor_name
        dfs.append(df_)
    df = pd.concat(dfs).reset_index().set_index(['factor', 'level'])
    return df

def package_prec_samples(prec_samples, factor_names):

    df = pd.DataFrame(prec_samples.T, index=factor_names)
    df.index = df.index.rename('factor')
    df.columns = df.columns.rename('iter')
    return df

In [ ]:
# select predictors

factor_names = ['stt', 'reg', 'eth', 'inc', 'age']

In [ ]:
# configure algorithm (hyperprior on the random effect prior precision is Wishart(prior_pseudo_tau, prior_guess_tau / prior_pseudo_tau))

n_samples = 1000
n_warmup = 100
prior_pseudo_tau = np.ones(len(factor_names))
prior_guess_tau = np.ones(len(factor_names))
seed = 0

In [ ]:
# create inputs

gg = pd.read_csv('demos/data/gg.csv')
n_true = gg.success
n_trials = gg.success + gg.failure
indices = gg.loc[:, factor_names].apply(lambda x: x.astype('category').cat.codes)
n_levels = indices.max(0).values + 1
rng = np.random.default_rng(seed)

In [ ]:
# sample

sampler = xfx.glm.binomial.sample_posterior(n_true.values, n_trials.values, n_levels, indices.values, prior_pseudo_tau, prior_guess_tau, ome=rng)
samples = [next(sampler) for _ in range(n_samples + n_warmup)][n_warmup:]

In [ ]:
# reformat samples for plotting

rfx_samples, prec_samples = package_samples(samples, factor_names)

In [ ]:
# random effects samples

rfx_samples.iloc[:5, :5]

In [ ]:
# prior precision samples

prec_samples.iloc[:5, :5]

In [ ]:
# plot traces for 'eth' random effects, grid and color by level

xfx.misc.plot.plot_traces(rfx_samples.loc['eth'], 'iter', 'level', 'level')

In [ ]:
# plot marginals for 'age' random effects, grid and color by level

xfx.misc.plot.plot_marginals(rfx_samples.loc['age'], 'level', 'level')

In [ ]:
# plot ACFs for prior precisions, grid and color by level

xfx.misc.plot.plot_acf(prec_samples, 'factor', 'factor')